In [1]:
import pymc4 as pm
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.compat.v1 import ConfigProto, InteractiveSession, OptimizerOptions

In [2]:
config = ConfigProto()
config.graph_options.optimizer_options.global_jit_level = (
    OptimizerOptions.ON_1)
config.intra_op_parallelism_threads = 1
config.inter_op_parallelism_threads = 1
sess = InteractiveSession(config=config)

# Linear Regression

In [3]:
# Logp calculation for linear regression 
@pm.model
def linreg(sd_prior='half_normal', n_points = 100):    
    # Define priors
    sigma = pm.HalfNormal('sigma', scale = 10)
    intercept = pm.Normal('Intercept', 0, scale=10)
    x_coeff = pm.Normal('weight', 0, scale = 5)
    x = np.linspace(-5, 5, n_points)
    
    # Define likelihood
    y = pm.Normal('y', loc=intercept + x_coeff * x, scale=sigma)
    
    
model = linreg.configure()

forward_sample = sess.run(model.forward_sample())

forward_sample

TypeError: _DistributionMeta object got multiple values for keyword argument 'scale'

In [ ]:
plt.scatter(np.linspace(-5, 5, 100), forward_sample['y'])

In [ ]:
func = model.make_log_prob_function()
sigma = tf.placeholder(tf.float32)
intercept = tf.placeholder(tf.float32)
x_coeff = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32, shape=(100,))
logp = func(sigma, intercept, x_coeff, y)


In [ ]:
feed_dict = {sigma: forward_sample['sigma'], 
            intercept: forward_sample['Intercept'], 
            x_coeff: forward_sample['weight'], 
            y: forward_sample['y']}

sess.run(logp, feed_dict=feed_dict)

In [ ]:
import pymc3 as pm3

In [ ]:
n_points = 100
with pm3.Model() as model3: 
    # Define priors
    sigma = pm3.HalfNormal('sigma', sd = 10, observed=forward_sample['sigma'])
    intercept = pm3.Normal('Intercept', 0, 10, observed=forward_sample['Intercept'])
    x_coeff = pm3.Normal('weight', 0, 5, observed=forward_sample['weight'])
    x = np.linspace(-5, 5, n_points)
    
    # Define likelihood
    y = pm3.Normal('y', intercept + x_coeff * x, sigma, observed=forward_sample['y'])
    
model3.logp()

# Centered-Eight 

In [ ]:
schools = np.array([28.,  8., -3.,  7., -1.,  1., 18., 12.])
sigma = tf.to_float(np.array([15., 10., 16., 11.,  9., 11., 10., 18.]))
n_points = 8

@pm.model
def centered_eight(n_points = n_points):    
    # Define priors
    mu = pm.Normal('mu', loc = 0, scale = 5)
    tau = pm.HalfCauchy('tau', loc = 0, scale = 5)
    theta = pm.Normal('theta', loc = tf.fill([8], mu.as_tensor()), scale = tf.fill([8], tau.as_tensor()))
    # Awkward syntax: Feature?
    
    # Define likelihood
    y = pm.Normal('y', loc = theta, scale = sigma)
    
model = centered_eight.configure()

forward_sample = sess.run(model.forward_sample())

forward_sample

In [ ]:
func = model.make_log_prob_function()
mu = tf.placeholder(tf.float32)
tau = tf.placeholder(tf.float32)
theta = tf.placeholder(tf.float32, shape=(n_points,))
y = tf.placeholder(tf.float32, shape=(n_points,))
logp = func(mu, tau, theta, y)

In [ ]:
feed_dict = {mu: forward_sample['mu'], 
            tau: forward_sample['tau'], 
            theta: forward_sample['theta'], 
            y: schools}

sess.run(logp, feed_dict=feed_dict)

In [ ]:
sigma = np.array([15., 10., 16., 11.,  9., 11., 10., 18.])
with pm3.Model() as model3: 
    # Define priors
    mu = pm3.Normal('mu', 0, sd = 5, observed=forward_sample['mu'])
    tau = pm3.HalfCauchy('Intercept', 5, observed=forward_sample['tau'])
    theta = pm3.Normal('weight', mu, tau, observed=forward_sample['theta'])
    
    # Define likelihood
    y = pm3.Normal('y', theta, sigma, observed=schools)
    
model3.logp()